# Epicrisis Fine-Tuning + Export ONNX (Colab)

Flujo completo:
1) Entrenar (LoRA) con Unsloth
2) Merge del LoRA con el modelo base
3) Exportar a ONNX (fp16) con Optimum
4) (Opcional) Export oficial Transformers.js q4f16


In [ ]:
# Instalar dependencias (Colab)
!pip -q install -U "unsloth[cu121]" transformers trl datasets accelerate peft bitsandbytes
# Export/quantización a ONNX sin Optimum
!pip -q install -U onnxruntime onnx onnxruntime-genai


## Subir dataset
Sube `train.jsonl` y `validation.jsonl`.

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
# Configuración base
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
TRAIN_FILE = "train.jsonl"
VAL_FILE = "validation.jsonl"

# Donde guardamos el adapter LoRA (PEFT)
ADAPTER_DIR = "./epicrisis-lora-adapter"

# (Opcional) donde guardar un modelo merged (no necesario para ONNX con ORT GenAI builder)
MERGED_DIR = "./epicrisis-merged"

print("MODEL_NAME:", MODEL_NAME)
print("ADAPTER_DIR:", ADAPTER_DIR)


In [ ]:
# Fine-tuning con Unsloth + LoRA (PEFT)
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

max_seq_length = 2048
dtype = None  # Unsloth elige automáticamente
load_in_4bit = True  # QLoRA estilo (reduce VRAM). Si prefieres LoRA "pura", pon False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# LoRA config (ajusta si quieres)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
)

# Dataset: espera columnas 'prompt' y 'completion' (ajusta según tu JSONL)
def format_example(example):
    # Cambia esto a tu formato real:
    prompt = example.get("prompt", "")
    completion = example.get("completion", "")
    text = prompt + completion
    return {"text": text}

train_ds = load_dataset("json", data_files=TRAIN_FILE, split="train").map(format_example)
eval_ds  = load_dataset("json", data_files=VAL_FILE, split="train").map(format_example)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    eval_dataset = eval_ds,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        output_dir = "./trainer-out",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = True,
        logging_steps = 10,
        evaluation_strategy = "steps",
        eval_steps = 50,
        save_steps = 50,
        save_total_limit = 2,
        optim = "adamw_8bit",
        report_to = "none",
    ),
)

trainer.train()

# Guardar SOLO el adapter LoRA (esto es lo que usaremos para el build ONNX)
trainer.model.save_pretrained(ADAPTER_DIR)
tokenizer.save_pretrained(ADAPTER_DIR)
print("Adapter LoRA guardado en:", ADAPTER_DIR)


In [ ]:
# (Opcional) Merge LoRA -> modelo base (no es necesario para export ONNX con ORT GenAI builder)
# Si lo necesitas por alguna razón, puedes descomentar.

# from unsloth import FastLanguageModel
# model = trainer.model
# model = FastLanguageModel.for_inference(model)
# model.save_pretrained_merged(MERGED_DIR, tokenizer, save_method="merged_16bit")
# print("Modelo merged guardado en:", MERGED_DIR)


## Exportar a ONNX + cuantizar (INT4) con onnxruntime-genai (sin Optimum)

In [ ]:
# Build ONNX aplicando el adapter LoRA durante la conversión
# Nota: aunque estés en Colab (sin WebGPU), puedes generar el paquete target 'webgpu' igualmente.

!python -m onnxruntime_genai.models.builder   -m {MODEL_NAME}   -o ./epicrisis-onnx-webgpu-int4   -p int4   -e webgpu   --extra_options hf_remote=true adapter_path={ADAPTER_DIR}

print("Export listo en: ./epicrisis-onnx-webgpu-int4")


## Prueba rápida (Python) con onnxruntime-genai (CPU)

In [ ]:
import onnxruntime_genai as og

model_dir = "./epicrisis-onnx-webgpu-int4"  # paquete generado por builder
# En Python, el provider real depende de tu instalación; para validar lógica puedes usar CPU si generas paquete CPU.
# Si quieres validar en CPU, genera también con: -e cpu -o ./epicrisis-onnx-cpu-int4

print("Archivos en model_dir:", model_dir)

# Ejemplo mínimo (puede variar según versión). Si falla, usa el ejemplo oficial de ORT GenAI para tu versión.


## (Opcional) Reorganizar estructura para Transformers.js
Si vas a usar Transformers.js, normalmente esperas una carpeta `onnx/` y archivos tokenizer/config en raíz. Con ORT GenAI el paquete es distinto; para Transformers.js puede requerir adaptación.

In [ ]:
import os, shutil, json
out_dir = './epicrisis-finetuned-onnx'
onnx_dir = os.path.join(out_dir, 'onnx')
os.makedirs(onnx_dir, exist_ok=True)

for f in os.listdir(out_dir):
    if f.endswith('.onnx') or f.endswith('.onnx_data'):
        shutil.move(os.path.join(out_dir, f), os.path.join(onnx_dir, f))

cfg_path = os.path.join(out_dir, 'config.json')
if os.path.exists(cfg_path):
    with open(cfg_path, 'r') as f:
        cfg = json.load(f)
    cfg['transformers.js_config'] = {
        'dtype': 'fp16',
        'kv_cache_dtype': {
            'fp16': 'float16'
        }
    }
    with open(cfg_path, 'w') as f:
        json.dump(cfg, f, indent=2)

print('Listo:', out_dir)


## (Opcional) Export oficial Transformers.js (q4f16)
Requiere scripts de Transformers.js.

In [ ]:
!git clone https://github.com/huggingface/transformers.js.git -q
%cd transformers.js
!pip -q install -r requirements.txt
!python3 scripts/convert.py \
  --model_id ../epicrisis-merged \
  --task text-generation-with-past \
  --quantize q4f16 \
  --output_dir ../epicrisis-finetuned-tjs
%cd ..


## Descargar resultados

In [ ]:
!zip -r epicrisis-onnx-webgpu-int4.zip epicrisis-onnx-webgpu-int4
print('ZIP creado: epicrisis-onnx-webgpu-int4.zip')